# Mount Drive and Install Dependencies

In [ ]:
#@title connect google drive folder

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/group_project/Roberta/

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

!pip install transformers


Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/group_project/Roberta/'
/content


In [ ]:
#@title Packages and Dependencies
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.optim import AdamW, lr_scheduler

import tqdm, pickle, csv, os, json, math, re
from typing import List, Optional, Union
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
transformers.logging.set_verbosity_error()
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## Translate emojies encodings to words and export to txt files

In [ ]:
# emoji_dict = './Emoji_Dict.p'
# with open(emoji_dict, 'rb') as fp:
#     Emoji_Dict = pickle.load(fp)
# Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convert_emojis_to_word(text):
#     for emot in Emoji_Dict:
#         text = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), text)
#     return text

# train_sentences = []
# with open('../data/train_emoji.txt', 'r', encoding = 'utf-8') as handle:
#     for line in handle:
#         train_sentences.append(line)

# with open('../data/train_emoji_decode.txt', "w", encoding = "utf-8-sig") as output:
#     for sentence in train_sentences:
#         sentence = convert_emojis_to_word(sentence)
#         output.write(sentence)

# test_sentences = []
# with open('../data/test.txt', 'r', encoding = 'utf-8') as handle:
#     for line in handle:
#         test_sentences.append(line)

# with open('../data/test_emoji_decode.txt', "w", encoding = "utf-8-sig") as output:
#     for sentence in test_sentences:
#         sentence = convert_emojis_to_word(sentence)
#         output.write(sentence)

In [ ]:
#@title PreProcessing
class InputExample(object):
    
    def __init__(self, guid, sentence, label):
        self.guid = guid
        self.sentence = sentence
        self.label = label
        
    def __str__(self):
        return "Guid: " + self.guid + "\nSentence:" + self.sentence + "\nLabel: " + self.label

class InputFeatures(object):

    def __init__(self, input_ids, attention_mask, label=None, token_type_ids = None):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label = label
        if token_type_ids:
            self.token_type_ids = token_type_ids

    def __str__(self):
        return "Input Ids: " + str(self.input_ids) + "\nAttention Mask:" + str(self.attention_mask) + "\nLabels: " + str(self.label)
        
class PreProcessor():
    def __init__(self):
        self.model_args = {'max_seq_length':64, 'verbose':False}

    def set_model_arg(self, key, value):
        self.model_args[key] = value
    
    def get_model_arg(self, key):
        return self.model_args.get(key, None)

    def save(self, path):
        f = open(path, 'wb')
        pickle.dump(self, f)
        f.close()

    def load(self, path):
        f = open(path, 'rb')
        proc = pickle.load(f)
        f.close()
        return proc

    def _read_txt(self, input_file):
        sentences = []
        with open(input_file, 'r', encoding = 'utf-8') as handle:
            for line in handle:
                sentences.append(line)
        return sentences

    def get_train_examples(self, data_dir, extra_data = False):
        if extra_data:
            return
        return self._create_examples(
            self._read_txt(os.path.join(data_dir, "train_emoji_decode.txt")))

    # def get_dev_examples(self, data_dir):
    #     return self._create_examples(
    #         self._read_txt(os.path.join(data_dir, "dev.jsonl")))
    
    def get_test_examples(self, data_dir):
        return self._create_examples(
            self._read_txt(os.path.join(data_dir, "test_emoji_decode.txt")))

    def _create_examples(self, lines):
        examples = []
        for (i, line) in enumerate(lines):
            if i != 0:
                features = line.split('\t')
                sentence = features[2]
                label = features[1]
                guid = features[0]
                examples.append(InputExample(guid = guid, sentence = sentence, label = int(label)))
        return examples

    def convert_example_to_feature(self, tokenizer, examples):
        max_length = proc.get_model_arg("max_seq_length")
        features = []

        if type(tokenizer) == type(RobertaTokenizer.from_pretrained('roberta-base')):
            for example in examples:
                inputs = tokenizer(example.sentence, add_special_tokens=True, max_length=max_length, padding = 'max_length')
                feature = InputFeatures(input_ids = inputs['input_ids'], attention_mask = inputs["attention_mask"], label = example.label)
                features.append(feature)

        else:
            for example in examples:
                inputs = tokenizer(example.sentence, add_special_tokens=True, max_length=max_length, padding = 'max_length')
                feature = InputFeatures(input_ids = inputs['input_ids'], attention_mask = inputs["attention_mask"],
                                        token_type_ids = inputs['token_type_ids'], label = example.label)
                features.append(feature)

        return features

    def convert_feature_to_dataset(self, features):
        # Convert to Tensors and build dataset
        
        if not hasattr(features[0], 'token_type_ids'):
            all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
            all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
            all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
            dataset = TensorDataset(all_input_ids, all_attention_mask, all_labels)
        else:
            all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
            all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
            all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
            all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
            dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

        return dataset
    
    def get_dataloader(self, features, batch_size, is_test=False, drop_last=True):
        dataset = self.convert_feature_to_dataset(features)
        dataset_sampler = SequentialSampler(dataset) if is_test else RandomSampler(dataset)
        dataloader = DataLoader(dataset, sampler=dataset_sampler, batch_size=batch_size, drop_last=drop_last)
        return dataloader
    
    def get_data_iter(self, features, batch_size, is_test=False, drop_last=True):
        dataloader = self.get_dataloader(features, batch_size, is_test=is_test, drop_last=drop_last)
        return iter(dataloader)

In [ ]:
#@title utility functions

def check_gpu():
    # torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
    is_cuda = torch.cuda.is_available()

    # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
    if is_cuda:
        device = torch.device("cuda")
        print("GPU is available")
    else:
        device = torch.device("cpu")
        print("GPU not available, CPU used")
    return device

def set_seed(seed=42):
    # random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model

def test_performance(y_true, y_pred):

    # Modify F1-score calculation depending on the task
    score = metrics.f1_score(y_true, y_pred, pos_label=1)
    p = metrics.precision_score(y_true, y_pred, pos_label=1)
    r = metrics.recall_score(y_true, y_pred, pos_label=1)
    acc = metrics.accuracy_score(y_true, y_pred)

    print('\n')
    print ("F1-score Task", score)
    print ("Precision Task", p)
    print ("Recall Task", r)
    print ("Accuracy Task", acc)

    false_index = [i for i, x in enumerate(y_pred == y_true) if not x]

    return false_index


In [ ]:
#@title Train
def train(model, tokenizer, proc, device, early_stopping = False):

    # fetch hyper-parameters
    batch_size = proc.get_model_arg("batch_size")
    n_epochs = proc.get_model_arg('n_epochs')
    learning_rate = proc.get_model_arg("learning_rate")
    adam_epsilon = proc.get_model_arg('adam_epsilon')
    weight_decay = proc.get_model_arg('weight_decay')
    verbose = proc.get_model_arg("verbose")
    log_step = proc.get_model_arg("log_step")
    checkpoint_path = proc.get_model_arg("checkpoint_path")
    max_seq_length = proc.get_model_arg("max_seq_length")
    clip = proc.get_model_arg("clip")
    data_path = proc.get_model_arg("dataset_path")

    # prepare training dataset
    examples = proc.get_train_examples(data_path)
    features = proc.convert_example_to_feature(tokenizer, examples)
    data_iter = proc.get_data_iter(features, batch_size)

    # training steps in each epoch
    examples_total_num = len(features)
    max_steps = math.ceil(float(examples_total_num)/batch_size)
    t_total = max_steps * n_epochs
    max_val_acc = 0 ## For early stopping

    # Define Loss, Optimizer
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    scheduler = lr_scheduler.LinearLR(optimizer, total_iters=t_total)

    epoch_loss = {'train_loss':[], 'val_accuracy':[]}

    if type(tokenizer) == type(bert_tokenizer):
        flag = True
    else:
        flag = False

    # train!
    for epoch in range(n_epochs):
        total_loss = 0.0
        pbar = tqdm.tqdm(range(max_steps))
        for step in pbar:
            model.train()
            optimizer.zero_grad() # Clears existing gradients from previous epoch
            # prepare inputs
            try:
                batch = next(data_iter)
            except StopIteration:
                data_iter = proc.get_data_iter(features, batch_size)
                batch = next(data_iter)

            batch = tuple(t.to(device) for t in batch)
            if flag:
                inputs = {'input_ids':      batch[0],
                            'attention_mask': batch[1],
                            'token_type_ids' : batch[2],
                            'labels':         batch[3]}
            else:
                inputs = {'input_ids':      batch[0],
                            'attention_mask': batch[1],
                            'labels':         batch[2]}

            outputs = model(**inputs)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
            loss.backward()
            # clip gradients to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            
            total_loss += loss.item()
            if step%log_step==0:
                _, val_acc = evaluate(model, tokenizer, proc, device)
                print("step: {}/{}, Train Loss: {:.4f}, Val Acc: {:.3f}".format(step, max_steps, loss.item(), val_acc))
        
        _, val_acc = evaluate(model, tokenizer, proc, device)
        print("epoch: {}/{}, Train Loss: {:.4f}, Val Accuracy: {}\n".format(epoch, n_epochs, total_loss/max_steps, val_acc))
        if early_stopping:
            if val_acc > max_val_acc:
                print('Improved Model saved at,', checkpoint_path)
                save_model(model, checkpoint_path)
                max_val_acc = val_acc

        epoch_loss['train_loss'].append(total_loss/max_steps)
        epoch_loss['val_accuracy'].append(val_acc)

    # if early_stopping:
    #     plt.figure(figsize = (12,6))
    #     plt.subplot(1,2,1)
    #     plt.plot(epoch_loss['train_loss'], color = 'red')
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Train Loss')
    #     plt.title('Training Loss')
    #     plt.subplot(1,2,2)
    #     plt.plot(epoch_loss['val_accuracy'], color = 'blue')   
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Validation Accuracy')
    #     plt.title('Validation Accuracy') 
    #     plt.show()

    if not early_stopping:
        print('Only save final model')
        save_model(model, checkpoint_path)
    
    return epoch_loss

In [ ]:
#@title Evaluate on dev set
def evaluate(model, tokenizer, proc, device):
    # fetch hyper-parameters
    batch_size = proc.get_model_arg("batch_size")
    n_epochs = proc.get_model_arg('n_epochs')
    learning_rate = proc.get_model_arg("learning_rate")
    adam_epsilon = proc.get_model_arg('adam_epsilon')
    weight_decay = proc.get_model_arg('weight_decay')
    verbose = proc.get_model_arg("verbose")
    log_step = proc.get_model_arg("log_step")
    checkpoint_path = proc.get_model_arg("checkpoint_path")
    max_seq_length = proc.get_model_arg("max_seq_length")
    clip = proc.get_model_arg("clip")
    data_path = proc.get_model_arg("dataset_path")
    
    # prepare dataset

    correct_counter = 0
    examples = proc.get_test_examples(data_path)
    sample_size = len(examples)
    features = proc.convert_example_to_feature(tokenizer, examples)
    eval_dataloader = proc.get_dataloader(features, batch_size, is_test = True,
                                          drop_last = False)

    all_predictions = []

    if type(tokenizer) == type(bert_tokenizer):
        flag = True
    else:
        flag = False

    model.eval()
    for batch in eval_dataloader:
        
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            if flag:
                inputs = {'input_ids':      batch[0],
                            'attention_mask': batch[1],
                            'token_type_ids' : batch[2],
                            'labels':         batch[3]}
            else:
                inputs = {'input_ids':      batch[0],
                            'attention_mask': batch[1],
                            'labels':         batch[2]}
            outputs = model(**inputs)
            predictions = outputs.logits.argmax(dim=-1)
            correct_counter += (predictions == inputs['labels']).sum()
        all_predictions.extend(list(predictions.clone().detach().to('cpu')))
    accuracy = correct_counter/sample_size
    # print('Validation Accuracy:', accuracy)

    return np.array(all_predictions), accuracy

# Training and Prediction

## Base HyperParameters:

proc.set_model_arg('batch_size', 64) <br>
proc.set_model_arg('max_seq_length', 128) <br>
proc.set_model_arg('learning_rate', 2e-5) <br>
proc.set_model_arg('n_epochs', 20) <br>
proc.set_model_arg('warmup_steps', 0.06) <br>
proc.set_model_arg('weight_decay', 0.1) <br>
proc.set_model_arg('adam_epsilon', 1e-8) <br>
proc.set_model_arg('clip', 1) <br>
proc.set_model_arg('log_step', 128) <br>
proc.set_model_arg('init_seed', 42) <br>

In [ ]:
#@title Setup Preprocessor with hyperparameters settings
# preprare dataset and hyper-parameters for training
proc = PreProcessor()

# hyper-parameters for data
proc.set_model_arg('batch_size', 64)
proc.set_model_arg('max_seq_length', 128)

# hyper-parameters for model
proc.set_model_arg('learning_rate', 2e-5)
proc.set_model_arg('n_epochs', 20)
proc.set_model_arg('warmup_steps', 0.06)
proc.set_model_arg('weight_decay', 0.1)
proc.set_model_arg('adam_epsilon', 1e-8)
proc.set_model_arg('clip', 1)

# arguments for reproduction
proc.set_model_arg('log_step', 128)
proc.set_model_arg('verbose', True)    # if log details
proc.set_model_arg('init_seed', 42)
proc.set_model_arg('checkpoint_path', "./model_checkpoint/roberta.bin")
proc.set_model_arg('dataset_path', "./data/")

# save proc
arg_path = "./procs/proc.dat"
proc.save(arg_path)


In [ ]:
#@title Bert-Base-Uncased
proc.set_model_arg('batch_size', 64)
proc.set_model_arg('checkpoint_path', "./model_checkpoint/bert-base.bin")
pre_trained_model = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(pre_trained_model)
model = BertForSequenceClassification.from_pretrained(pre_trained_model)

# get parameters from preprocessor
init_seed = proc.get_model_arg('init_seed')
proc.set_model_arg('verbose', False)

device = check_gpu()
set_seed(init_seed)

# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)
epoch_loss = train(model, tokenizer, proc, device,
                   early_stopping = True)

GPU is available


  2%|▏         | 1/60 [00:07<06:55,  7.04s/it]

step: 0/60, Train Loss: 0.7881, Val Acc: 0.398


100%|██████████| 60/60 [00:48<00:00,  1.24it/s]


epoch: 0/20, Train Loss: 0.6715, Val Accuracy: 0.6313775181770325

Improved Model saved at, ./model_checkpoint/bert-base.bin


...

epoch: 18/20, Train Loss: 0.0254, Val Accuracy: 0.6823979616165161



  2%|▏         | 1/60 [00:05<05:35,  5.68s/it]

step: 0/60, Train Loss: 0.1058, Val Acc: 0.676


100%|██████████| 60/60 [00:47<00:00,  1.27it/s]


epoch: 19/20, Train Loss: 0.0279, Val Accuracy: 0.6823979616165161



In [ ]:
#@title Bert-Large
proc.set_model_arg('batch_size', 32)
proc.set_model_arg('checkpoint_path', "./model_checkpoint/bert-large.bin")
pre_trained_model = "bert-large-uncased"

tokenizer = BertTokenizer.from_pretrained(pre_trained_model)
model = BertForSequenceClassification.from_pretrained(pre_trained_model)

# get parameters from preprocessor
init_seed = proc.get_model_arg('init_seed')
proc.set_model_arg('verbose', False)

device = check_gpu()
set_seed(init_seed)

# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)
epoch_loss = train(model, tokenizer, proc, device,
                   early_stopping = True)

GPU is available


  1%|          | 1/120 [00:12<25:29, 12.85s/it]

step: 0/120, Train Loss: 1.2464, Val Acc: 0.397


100%|██████████| 120/120 [02:39<00:00,  1.33s/it]


epoch: 0/20, Train Loss: 0.6651, Val Accuracy: 0.7053571343421936

Improved Model saved at, ./model_checkpoint/bert-large.bin


...

epoch: 18/20, Train Loss: 0.0362, Val Accuracy: 0.7436224222183228



  1%|          | 1/120 [00:12<25:10, 12.69s/it]

step: 0/120, Train Loss: 0.0116, Val Acc: 0.745


100%|██████████| 120/120 [02:37<00:00,  1.31s/it]


epoch: 19/20, Train Loss: 0.0335, Val Accuracy: 0.730867326259613



In [ ]:
#@title Roberta-With-Pretrained-Weights
proc.set_model_arg('batch_size', 32)
proc.set_model_arg('checkpoint_path', "./model_checkpoint/roberta_pretrained.bin")
pre_trained_model = "cardiffnlp/twitter-roberta-base-irony"

tokenizer = RobertaTokenizer.from_pretrained(pre_trained_model)
model = RobertaForSequenceClassification.from_pretrained(pre_trained_model)

# get parameters from preprocessor
init_seed = proc.get_model_arg('init_seed')
proc.set_model_arg('verbose', False)

device = check_gpu()
set_seed(init_seed)

# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)
epoch_loss = train(model, tokenizer, proc, device,
                   early_stopping = True)

GPU is available


  1%|          | 1/120 [00:05<10:50,  5.47s/it]

step: 0/120, Train Loss: 0.2304, Val Acc: 0.737


100%|██████████| 120/120 [00:50<00:00,  2.35it/s]


epoch: 0/20, Train Loss: 0.2494, Val Accuracy: 0.7040815949440002

Improved Model saved at, ./model_checkpoint/roberta_pretrained.bin


...

epoch: 18/20, Train Loss: 0.0113, Val Accuracy: 0.7474489808082581



  1%|          | 1/120 [00:05<10:07,  5.11s/it]

step: 0/120, Train Loss: 0.0000, Val Acc: 0.747


100%|██████████| 120/120 [00:50<00:00,  2.37it/s]


epoch: 19/20, Train Loss: 0.0190, Val Accuracy: 0.7487244606018066



In [ ]:
#@title Roberta-large-from-scratch
pre_trained_model = "roberta-large"
proc.set_model_arg('batch_size', 32)
proc.set_model_arg('checkpoint_path', "./model_checkpoint/roberta-large.bin")

tokenizer = RobertaTokenizer.from_pretrained(pre_trained_model)
model = RobertaForSequenceClassification.from_pretrained(pre_trained_model)

# get parameters from preprocessor
init_seed = proc.get_model_arg('init_seed')
proc.set_model_arg('verbose', False)

device = check_gpu()
set_seed(init_seed)

# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)
epoch_loss = train(model, tokenizer, proc, device,
                   early_stopping = True)

GPU is available


  1%|          | 1/120 [00:12<25:42, 12.96s/it]

step: 0/120, Train Loss: 0.6979, Val Acc: 0.603


100%|██████████| 120/120 [02:38<00:00,  1.32s/it]


epoch: 0/20, Train Loss: 0.4503, Val Accuracy: 0.605867326259613

Improved Model saved at, ./model_checkpoint/roberta-large.bin


...

epoch: 18/20, Train Loss: 0.0361, Val Accuracy: 0.625



  1%|          | 1/120 [00:12<25:09, 12.69s/it]

step: 0/120, Train Loss: 0.0002, Val Acc: 0.625


100%|██████████| 120/120 [02:39<00:00,  1.33s/it]


epoch: 19/20, Train Loss: 0.0413, Val Accuracy: 0.6211734414100647



# Models Evaluation and Error Analysis

In [ ]:
## Get the test labels
examples = proc.get_test_examples(proc.get_model_arg('dataset_path'))
test_labels = []
for example in examples:
    test_labels.append(example.label)

test_labels = np.array(test_labels)

## Bert-Base-Uncased

In [ ]:
model_path = "./model_checkpoint/bert-base.bin"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model = load_model (model, model_path)
model.to(device)

predictions, _ = evaluate(model, tokenizer, proc, device)

# Evaluate Test Result
bert_base_false_index = test_performance(test_labels, predictions)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).




F1-score Task 0.6388443017656501
Precision Task 0.6378205128205128
Recall Task 0.639871382636656
Accuracy Task 0.7130102040816326


## Bert-Large

In [ ]:
model_path = "./model_checkpoint/bert-large.bin"
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")
model = load_model (model, model_path)
model.to(device)

predictions, _ = evaluate(model, tokenizer, proc, device)

# Evaluate Test Result
bert_large_false_index = test_performance(test_labels, predictions)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).




F1-score Task 0.7202295552367288
Precision Task 0.6502590673575129
Recall Task 0.8070739549839229
Accuracy Task 0.7512755102040817


## Roberta-Large

In [ ]:
model_path = "./model_checkpoint/roberta-large.bin"
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model = RobertaForSequenceClassification.from_pretrained("roberta-large")
model = load_model (model, model_path)
model.to(device)

predictions, _ = evaluate(model, tokenizer, proc, device)

# Evaluate Test Result
roberta_large_false_index = test_performance(test_labels, predictions)



F1-score Task 0.42028985507246375
Precision Task 0.8446601941747572
Recall Task 0.2797427652733119
Accuracy Task 0.6938775510204082


## Roberta with pretrained weights

In [ ]:
model_path = "./model_checkpoint/roberta_pretrained.bin"
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-irony")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-irony")
model = load_model (model, model_path)
model.to(device)

predictions, _ = evaluate(model, tokenizer, proc, device)

# Evaluate Test Result
roberta_large_false_index = test_performance(test_labels, predictions)



F1-score Task 0.6613545816733067
Precision Task 0.8691099476439791
Recall Task 0.5337620578778135
Accuracy Task 0.7831632653061225
